<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/bugfix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
start_date.value.strftime('%Y-%m-%d')
end_date.value.strftime('%Y-%m-%d')
limit.value

In [16]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata
import ipywidgets as w
from IPython.display import display
import datetime
engine = create_engine(userdata.get('supabase'), client_encoding='utf8', )


@w.interact_manual(start_date=w.DatePicker(description='Start Date', disabled=False, value = datetime.date(2021,9,1)),
                  end_date = w.DatePicker(description='End Date', disabled=False, value = datetime.date(2021,9,21)),
                  limit = w.IntText(description='Limit', value=10)
                  )
def select(start_date, end_date, limit):
    SQL_TEMPLATE_QUERY_LINECHART = """
    WITH t AS (
        SELECT
            query,
            ROW_NUMBER() OVER (PARTITION BY date_part('hour', ts) ORDER BY COUNT(*) DESC) AS rnb
        FROM
            vizro.yandex_data yd
        WHERE
            ts BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY query, date_part('hour', ts)
    )
    SELECT
        platform,
        date_part('hour', ts)::INT AS hour,
        query,
        COUNT(*)
    FROM
        vizro.yandex_data yd
    WHERE
        query IN (SELECT DISTINCT query FROM t WHERE rnb < 6)
    GROUP BY
        platform, date_part('hour', ts), query
        {limit}
    """.format(start_date=start_date, end_date=end_date, limit=f'LIMIT {limit}' if limit>0 else '')
    sql = text(SQL_TEMPLATE_QUERY_LINECHART)

    return pd.read_sql(sql, engine)

interactive(children=(DatePicker(value=datetime.date(2021, 9, 1), description='Start Date'), DatePicker(value=…